In [ ]:
from urlparse import urljoin
from bs4 import BeautifulSoup
import requests

In [ ]:
# Lists of charts to find on StatisticalAtlas

pd = ["Population Density by Block Group"]
# aas = ["Detailed Age and Sex", "Detailed Sex Ratio"]
aas = ["Age Structure"]
rae = ["Race and Ethnicity"]
ht = ["Household Types"]
ms = ["Marital Status"]
no = ["Citizenship and National Origin"]
# hi = ["Household Income Percentiles"]
hi = ["Median Household Income by Race"]
es = ["Employment Status"]
# fs = ["On Food Stamps by Race"]
fs = ["Most Food Stamps by Tract"]
ea = ["Educational Attainment", "Lacking High School Diploma By Race", "College Graduates By Race"]
ee = ["School Enrollment", "School Enrollment By Age"]
link = "https://statisticalatlas.com/block-group/Illinois/Cook-County/010503-1/Age-and-Sex"
# link = "https://statisticalatlas.com/block-group/Illinois/Cook-County/834800-2/Race-and-Ethnicity"
# link = "https://statisticalatlas.com/block-group/Illinois/Cook-County/834800-2/Household-Types"
# link = "https://statisticalatlas.com/block-group/Illinois/Cook-County/834800-2/Marital-Status"
resp = requests.get(link)
soup = BeautifulSoup(resp.content)


otls = get_charts(soup, aas)
# pops = get_pop(soup)
for tp in otls:
    print tp[0]
    print "\n"*3
    print tp[1]
    print "\n"*10

In [ ]:
# def check_has(sp):
#     lizt = sp.find_all("h2")
#     for h2 in lizt:
#         if h2.text == "Where Are the Charts?":
#             return False
#     return True

# Custom scraper for pop density

def get_pop(sp):
    lst = get_charts(sp, ["Population Density by Block Group"])
    if (lst == "ERROR"):
        return "ERROR"
    for tp in lst:
        if (len(tp[0]) != len(tp[1])):
            return "ERROR"
        return (tp[0],tp[1])    
    
def tilc(string, cr):
    ans = ""
    for c in string:
        if c == cr:
            break
        ans += c
    return ans

# scrapes data from charts

def get_charts(sp, title_list):
#     lizt = sp.find_all("div", "row figure-row gray-rounded-box")
    lizt = sp.find_all("div", "figure-container")
    outlist = []
    gdct = 0
    for i in lizt:
        title = i.find("h4")
        if (title == None or (not title.has_attr("class")) or title["class"][0] != "figure-title"):
            continue
#         print title.text
        xlist = []
        ylist = []
        
        if (title_list[0] == "Population Density by Block Group" or title_list[0] == "Most Food Stamps by Tract"):
            l1 = i.find_all('a')
            for j in l1:
                if j.find("text") and j.find("text").has_attr("font-style"):
                    xlist.append(j.find("text").text)
                    
        l2 = i.find_all('g')
        
        for j in l2:
            if j.find("title") and not j.find("line"):
                ylist.append(j.find("title").text)
            if j.find("text") and j.find("text").has_attr("font-style"):
                xlist.append(j.find("text").text)
        if (tilc(title.text,'#')[1:] in title_list):
            outlist.append((xlist, ylist))
            gdct += 1
    print gdct
    if (gdct != len(title_list)):
        return "ERROR"
    return outlist

In [ ]:
# Main scraping operations here

plist = []
badlist = []
# sufs = ["Population"]
# sufs = ["Marital-Status"]
# sufs = ["Household-Income"]
sufs = ["Age-and-Sex"]

bgf = open("/Users/ryu/Desktop/bglist.txt", 'r')
N = int(bgf.readline()[:-1])

S = requests.Session()
for i in range(N):
    name = bgf.readline()[:-1]
    print "Processing:", name
    base = "https://statisticalatlas.com/block-group/Illinois/Cook-County/"+name[:-1]+'-'+name[-1]+'/'
    for sf in sufs:
        link = base + sf
        resp = S.get(link)
        sp = BeautifulSoup(resp.content)
        lx2 = get_charts(sp, aas)
#         lx2 = get_pop(sp)
        if lx2 == "ERROR":
            badlist.append(name)
            print "ERROR"
            continue
        print "SUCCESS"
        plist.append((name,lx2))

In [ ]:
# Format and print out data

bins = [u'35+', u'25-34 ', u'20-24 ', u'18-19', u'15-17', u'10-14', u'5-9', u'3-4']
# 0-12, 13-14, 15-17, 18-18, 19-19, 20-20, 21-21, 22-22
print str(len(plist))
for pl in plist:
    print pl[0]
    tpt = [0 for i in range(8)]
    for i,a in enumerate(pl[1][1][1]):
        if (i%2):
            tpt[bins.index(pl[1][1][0][i/2])] = a[:-1]
    for i in tpt:
        print i,
#     if len(pl[1][1][0])*2 != len(pl[1][1][1]) or len(pl[1][1][0]) != 8:
#         print "BOOHBAH"
#     for i,a in enumerate(pl[1][1][1]):
#         if (i%2):
#             print a[:-1],
    print